In [1]:
import numpy as np
import plotly.graph_objects as go



In [2]:
def rotation_matrix_x(phi):
    """Generate rotation matrix for a roll (rotation about the x-axis)"""
    phi = (phi / 180) * np.pi
    c, s = np.cos(phi), np.sin(phi)
    return np.array([[1, 0, 0], 
                     [0, c, -s], 
                     [0, s, c]])

def rotation_matrix_y(theta):
    """Generate rotation matrix for a pitch (rotation about the y-axis)"""
    theta = (theta / 180) * np.pi
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, 0, s], 
                     [0, 1, 0], 
                     [-s, 0, c]])

def rotation_matrix_z(psi):
    """Generate rotation matrix for a yaw (rotation about the z-axis)"""
    psi = (psi / 180) * np.pi
    c, s = np.cos(psi), np.sin(psi)
    return np.array([[c, -s, 0], 
                     [s, c, 0], 
                     [0, 0, 1]])

In [3]:
def rotation_matrix_to_prv(C):
    """
    Converts a rotation matrix to a Principal Rotation Vector (PRV).

    Args:
        C (np.array): A 3x3 rotation matrix.

    Returns:
        tuple: A PRV represented as (e_vector, phi_angle).
    """
    # Compute the angle phi from the trace of the rotation matrix
    trace_C = np.trace(C)
    phi = np.arccos((trace_C - 1) / 2)
    
    # Handle edge cases where phi is 0 or π
    if np.isclose(phi, 0) or np.isclose(phi, np.pi):
        
        # For phi=0, no rotation, the axis can be arbitrary, choose x-axis for simplicity
        # For phi=π, rotation by 180 degrees, find axis by identifying non-zero component
        e = np.array([1, 0, 0])  # Arbitrary axis, could also check for non-diagonal elements
    
    else:
        # Compute the unit vector e from the off-diagonal elements of the matrix C
        e = (1 / (2 * np.sin(phi))) * np.array([C[1, 2] - C[2, 1],
                                                C[2, 0] - C[0, 2],
                                                C[0, 1] - C[1, 0]])
        # Normalize the unit vector to ensure it's a valid unit vector
        e /= np.linalg.norm(e)

    # Ensure the angle phi is in the range [0, 2*pi)
    phi = np.mod(phi, 2 * np.pi)

    return e, phi

In [4]:
def prv_to_rotation_matrix(e, phi_deg):
    """
    Converts a Principal Rotation Vector (PRV) to a rotation matrix.

    Args:
        e (np.array)   : The unit vector of the PRV.
        phi_deg (float): The rotation angle of the PRV in degrees.

    Returns:
        np.array: A 3x3 rotation matrix.
    """
    # Convert the angle from degrees to radians
    phi_rad = np.radians(phi_deg)
    
    # Calculate the cosine and sine of the angle
    c_phi = np.cos(phi_rad)
    s_phi = np.sin(phi_rad)
    
    # Calculate the matrix Sigma
    Sigma = 1 - c_phi

    # Ensure e is a float array to avoid UFuncTypeError during in-place operations
    e = np.array(e, dtype=float)
    
    # Normalize e vector to ensure it's a valid unit vector
    e /= np.linalg.norm(e)
    
    # Decompose the unit vector into its components
    e1, e2, e3 = e
    
    # Construct the rotation matrix using the given formula
    C = np.array([[((e1**2)*Sigma + c_phi), (e1*e2*Sigma + e3*s_phi), (e1*e3*Sigma - e2*s_phi)],
                  [(e2*e1*Sigma - e3*s_phi), ((e2**2)*Sigma + c_phi), (e2*e3*Sigma + e1*s_phi)],
                  [(e3*e1*Sigma + e2*s_phi), (e3*e2*Sigma - e1*s_phi), ((e3**2)*Sigma + c_phi)]])

    return C

In [5]:
def setup_animation_controls(fig, frames):
    """
    Adds animation controls and configures layout settings for a Plotly figure.

    Args:
        fig (plotly.graph_objects.Figure): The figure to which the controls will be added.
        frames (list): List of animation frames to be included in the slider control.

    Note:
    - Animation Controls: This function sets up interactive controls that allow users to play through or step through the animation frames at their own pace. 
                          The 'Play' button starts the animation, while the slider allows users to jump to specific points in the animation.
    
    - Button Configuration: The 'Play' button is configured to start the animation immediately when clicked, using the 'immediate' mode. 
                            It ensures that the animation runs smoothly, redrawing each frame with a duration of 100 milliseconds.
    
    - Slider Mechanism: The slider below the animation includes steps corresponding to each frame created in the animation sequence. 
                        Users can move the slider to navigate to different frames, which is particularly useful for examining specific moments of the animation in detail.
    
    - Current Value Display: The slider also shows the current frame number as the animation plays, 
                             providing immediate visual feedback to users on their current position within the animation sequence.
    
    - Layout Configuration: The function also defines the visual layout of the 3D scene. 
                            It sets a fixed cube aspect for the axes to ensure that objects in the 3D space are displayed proportionally. 
                            The axes are configured to not auto-scale, maintaining consistent spatial references.
    
    - Scene Dimensions: The overall dimensions of the figure are set to ensure sufficient space for viewing the animation comfortably, enhancing the visual experience for the user.
    
    - Usability and Interactivity: By integrating these controls, the function enhances the usability and interactivity of the visualization, 
                                   making it a more effective tool for presentations or educational purposes where step-by-step analysis of movements is beneficial.

    """
    # Update the figure with animation controls
    fig.update_layout(updatemenus=[{"type": "buttons",
                                    "showactive": False,
                                    "y": 1.05,
                                    "x": 0.8,
                                    "xanchor": 'left',
                                    "yanchor": 'bottom',
                                    "buttons": [{"label": 'Play',
                                                 "method": 'animate',
                                                 "args": [None, {"frame": {"duration": 100, "redraw": True},
                                                                 "fromcurrent": True,
                                                                 "mode": 'immediate'}]}]}],
                      sliders=[{"steps": [{"method": 'animate',
                                           "args": [[f.name], {"mode": 'immediate', 
                                                               "frame": {"duration": 100, "redraw": True},
                                                               "fromcurrent": True}],
                                           "label": str(k)} for k, f in enumerate(frames)],
                                "x": 0.1,
                                "y": 0,
                                "currentvalue": {"visible": True, "prefix": 'Step: '}}])

    # Set additional layout settings for the 3D scene
    fig.update_layout(width=1000,
                      height=800,
                      template='presentation',
                      scene={"aspectmode": 'cube',
                             "xaxis": {"range": [-1, 1], "autorange": False},
                             "yaxis": {"range": [-1, 1], "autorange": False},
                             "zaxis": {"range": [-1, 1], "autorange": False}},
                      title='Euler Rotation Animation')

In [8]:
# Initial and final orientations
R_initial = np.eye(3)
R_final = rotation_matrix_z(45) @ rotation_matrix_y(45) @ rotation_matrix_z(45)

# Compute PRV from initial to final rotation matrices
axis, phi = rotation_matrix_to_prv(R_final @ R_initial.T)

# Visualization setup
fig = go.Figure()

# Add trace for the rotation axis
axis_length = 2  # Length to extend the rotation axis for visibility
fig.add_trace(go.Scatter3d(
    x=[-axis[0]*axis_length, axis[0]*axis_length],
    y=[-axis[1]*axis_length, axis[1]*axis_length],
    z=[-axis[2]*axis_length, axis[2]*axis_length],
    mode='lines',
    line=dict(width=6, color='yellow'),
    name='Rotation Axis'
))

frames = []
steps = 30
for step in range(steps + 1):
    interpolated_angle = phi * step / steps
    R_step = prv_to_rotation_matrix(axis, interpolated_angle)
    vectors = R_step @ R_initial

    frame_data = [
        go.Scatter3d(x=[0, vec[0]], y=[0, vec[1]], z=[0, vec[2]],
                     mode='lines+markers', line=dict(width=5),
                     marker=dict(size=4, color=['red', 'green', 'blue'][i]),
                     name=f'Rotating Axis {["X", "Y", "Z"][i]}')
        for i, vec in enumerate(vectors.T)
    ]
    frames.append(go.Frame(data=frame_data, name=str(step)))

# Add the first frame's data to initialize the figure
fig.add_traces(frames[0].data)

# Add controls and display
fig.frames = frames

fig.update_layout(
    updatemenus=[{
        "type": "buttons",
        "buttons": [{
            "label": "Play",
            "method": "animate",
            "args": [None, {"frame": {"duration": 100, "redraw": True}, "fromcurrent": True}]
        }],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }],
    sliders=[{
        "steps": [{"method": "animate", "args": [[f.name], {"mode": "immediate", "frame": {"duration": 100, "redraw": True}}], "label": str(k)} for k, f in enumerate(frames)],
        "active": 0,
        "currentvalue": {"visible": True, "prefix": "Rotation:"},
        "x": 0.1,
        "len": 0.9
    }]
)

# Additional layout settings
fig.update_layout(width=1000,
                  height=800,
                  template='presentation',
                  scene={"aspectmode": 'cube',
                         "xaxis": {"range": [-1, 1], "autorange": False},
                         "yaxis": {"range": [-1, 1], "autorange": False},
                         "zaxis": {"range": [-1, 1], "autorange": False}},
                  title='PRV Rotation Animation')

# Display the figure
fig.show()